# Experimento Sklearn

## Libs

In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NeighborhoodComponentsAnalysis

import pandas as pd
import numpy as np

In [2]:
#iris = load_iris(as_frame=True)
#X, y = iris.data, iris.target

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [3]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# Experimento TCC

In [4]:
#from multiprocessing import cpu_count

from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient

In [5]:
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
nca = NCA(optimization_func=PSO, max_iter=1000, k=5, cpu_count=None)#cpu_count())

In [8]:
nca.fit(X_train,y_train)
y_pred = nca.predict(X_test)

In [9]:
np.mean(y_pred == y_test)

0.9666666666666667

In [10]:
nca_grad = NCA(optimization_func=Gradient, max_iter=1000, k=5,cpu_count=None)#cpu_count())

In [11]:
nca_grad.fit(X_train, y_train)
y_pred_grad = nca_grad.predict(X_test)
np.mean(y_test == y_pred_grad)

0.9833333333333333